In [1]:
import pandas as pd
import json
from datetime import datetime
import pymysql

In [4]:
# Excel to JSON Conversion
def clean_data(value):
    """Cleans data by:
    - Stripping extra spaces
    - Replacing "NULL" or empty strings with None
    - Formatting dates correctly
    - Converting decimals properly
    """
    if isinstance(value, str):
        value = value.strip()  # Remove extra spaces
        if value.upper() == "NULL" or value == "":
            return None  # Convert "NULL" string to actual null (None)
        return value

    elif isinstance(value, pd.Timestamp) or isinstance(value, datetime):
        return value.strftime("%Y-%m-%d %H:%M:%S")  # Convert date format

    elif isinstance(value, float) or isinstance(value, int):
        return round(value, 2)  # Ensure decimals are properly formatted

    return value  # Return as is for other data types

# Load the Excel file
excel_file = "../_Dump data/EXCEL/sub-table.xlsx"  # Replace with your actual Excel file
df = pd.read_excel(excel_file, dtype=str)  # Read everything as string initially

# Clean column headers (remove extra spaces)
df.columns = [col.strip() for col in df.columns]

# Apply the cleaning function to each column using apply()
df = df.apply(lambda col: col.map(clean_data))

# Convert to JSON
json_data = df.to_json(orient="records", indent=4)

# Save JSON file
json_file = "../_Dump data/JSON/remarks output.json"
with open(json_file, "w", encoding="utf-8") as file:
    file.write(json_data)

print(f"Excel file '{excel_file}' converted to JSON and saved as '{json_file}' successfully!")

Excel file '../_Dump data/EXCEL/sub-table.xlsx' converted to JSON and saved as '../_Dump data/JSON/remarks output.json' successfully!


In [12]:
# data import json to mysql #++++++++++++# For Sub table
# Used at Category, Sub Category, FundSource, Contractor, remarks, remarks, Remark, Year
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="monitrack",
    cursorclass=pymysql.cursors.DictCursor
)

cursor = conn.cursor()

# Load JSON file
with open("../_Dump data/JSON/year output.json", "r") as json_file:
    data = json.load(json_file)

# Convert "None" (string) and empty values to None (Python None -> MySQL NULL)
for record in data:
    for key, value in record.items():
        if value in ["None", None, ""]:  
            record[key] = None

# Insert data into MySQL
sql = """
INSERT INTO home_year (
    `id`,
    `year`
) VALUES (
    %s, %s
);
"""

for record in data:
    cursor.execute(sql, (
        record.get("ID"),
        record.get("YEAR")))

# Commit changes and close connection
conn.commit()
conn.close()

print("✅ JSON imported into MySQL with NULL values correctly!")


✅ JSON imported into MySQL with NULL values correctly!


In [13]:
# Database connection #++++++++++++# For Main table
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="monitrack",
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

# Load JSON file
with open("../_Dump data/JSON/all output.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Function to clean data
def clean_value(value):
    """Cleans data by stripping spaces, converting 'NULL' to None, and handling empty values."""
    if isinstance(value, str):
        value = value.strip()
        return None if value.upper() == "NULL" or value == "" else value
    return value

# Convert all records
for record in data:
    for key in record.keys():
        record[key] = clean_value(record[key])

# Foreign key mapping (Example: Replace names with IDs)
def get_foreign_key_id(table, column, value):
    """Fetches the ID of a related record based on its name."""
    if value is None:
        return None
    cursor.execute(f"SELECT id FROM {table} WHERE {column} = %s", (value,))
    result = cursor.fetchone()
    return result["id"] if result else None

# Process records for foreign keys
for record in data:
    record["CATEGORY"] = get_foreign_key_id("home_category", "category", record.get("CATEGORY"))
    record["SUB CATEGORY"] = get_foreign_key_id("home_subcategory", "sub_category", record.get("SUB CATEGORY"))
    record["SOURCE OF FUND"] = get_foreign_key_id("home_fundsource", "fund", record.get("SOURCE OF FUND"))
    record["MUNICIPALITY"] = get_foreign_key_id("home_municipality", "municipality", record.get("MUNICIPALITY"))
    record["IMPLEMENTING OFFICE"] = get_foreign_key_id("home_office", "office", record.get("IMPLEMENTING OFFICE"))
    record["YEAR"] = get_foreign_key_id("home_year", "year", record.get("YEAR"))

# SQL Insert Query
sql = """
INSERT INTO home_project (
    project_number, project_name, project_ID, project_description, location, 
    category_id, fund_id, municipality_id, office_id, sub_category_id, year_id
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insert data
try:
    cursor.executemany(sql, [
        (
            record.get("NO"),
            record.get("PROJECT NAME"),
            record.get("PROJECT ID"),
            record.get("PROJECT DESCRIPTION"),
            record.get("LOCATION"),
            record.get("CATEGORY"),
            record.get("SOURCE OF FUND"),
            record.get("MUNICIPALITY"),
            record.get("IMPLEMENTING OFFICE"),
            record.get("SUB CATEGORY"),
            record.get("YEAR")
        )
        for record in data
    ])
    conn.commit()
    print("✅ JSON imported into MySQL successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error: {e}")
    conn.rollback()

# Close connection
conn.close()

✅ JSON imported into MySQL successfully!


In [16]:
# Database connection #++++++++++++# For Sub Main CONTRACT table
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="monitrack",
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

# Load JSON file
with open("../_Dump data/JSON/all output.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Function to clean data
def clean_value(value):
    """Cleans data by stripping spaces, converting 'NULL' to None, and handling empty values."""
    if isinstance(value, str):
        value = value.strip()
        return None if value.upper() == "NULL" or value == "" else value
    return value

# Clean all records
for record in data:
    for key in record.keys():
        record[key] = clean_value(record[key])

# Function to fetch contractor ID (considering both contractor name & TIN)
def get_contractor_id(contractor, tin_number):
    """Fetches contractor ID based on contractor name and TIN (does NOT insert if not found)."""
    if not contractor:
        return None  # No contractor provided

    # Check if the contractor with the given TIN exists
    if tin_number:
        cursor.execute(
            "SELECT id FROM home_contractor WHERE contractor = %s AND tin_number = %s",
            (contractor, tin_number)
        )
    else:
        cursor.execute(
            "SELECT id FROM home_contractor WHERE contractor = %s AND tin_number IS NULL",
            (contractor,)
        )
    
    result = cursor.fetchone()
    return result["id"] if result else None  # Return ID if found, otherwise None

# Function to fetch foreign key ID from another table
def get_foreign_key_id(table, column, value):
    """Fetches the ID of a related record based on its name."""
    if value is None:
        return None
    cursor.execute(f"SELECT id FROM {table} WHERE {column} = %s", (value,))
    result = cursor.fetchone()
    return result["id"] if result else None

# Process records for foreign keys
for record in data:
    record["GENERAL REMARKS"] = get_foreign_key_id("home_remark", "remark", record.get("GENERAL REMARKS"))

# SQL Insert Query
sql = """
INSERT INTO home_contract (
    project_cost, contract_cost, procurement, project_contractor, tin_number, quarter, project_id, remarks_id
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insert data
try:
    cursor.executemany(sql, [
        (
            record.get("PROJECT COST"),
            record.get("CONTRACT COST"),
            record.get("MODE OF PROCUREMENT"),
            record.get("PROJECT CONTRACTOR"),
            record.get("TIN NUMBER"),
            record.get("QUARTER"),
            record.get("NO"),
            record.get("GENERAL REMARKS")
        )
        for record in data
    ])
    conn.commit()
    print("✅ JSON imported into MySQL successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error: {e}")
    conn.rollback()

# Close connection
conn.close()

✅ JSON imported into MySQL successfully!


In [17]:
# Database connection #++++++++++++# For Sub Main PROJECTTIMELINE table
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="monitrack",
    cursorclass=pymysql.cursors.DictCursor
)

cursor = conn.cursor()

# Load JSON file
with open("../_Dump data/JSON/all output.json", "r") as json_file:
    data = json.load(json_file)

# Convert "None" (string) and empty values to None (Python None -> MySQL NULL)
for record in data:
    for key, value in record.items():
        if value in ["None", None, ""]:  
            record[key] = None

# Insert data into MySQL
sql = """
INSERT INTO home_projecttimeline (
    `cd`, `ntp_date`, `extension`, `target_completion_date`, 
    `revised_completion_date`, `date_completed`, `project_id` , `total_cost_Incurred_to_date`, `reason`
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s
);
"""

for record in data:
    cursor.execute(sql, (
        record.get("C.D"),
        record.get("NTP DATE"),
        record.get("NO. OF EXTENSION"),
        record.get("TARGET COMPLETION DATE"),
        record.get("REVISED COMPLETION DATE"),
        record.get("DATA COMPLETED"),
        record.get("NO"),
        record.get("TOTAL COST INCURED TO DATE"),
        record.get("REASON")
    ))

# Commit changes and close connection
conn.commit()
conn.close()

print("✅ JSON imported into MySQL with NULL values correctly!")

✅ JSON imported into MySQL with NULL values correctly!
